<a href="https://colab.research.google.com/github/RodrigodeFeo/8DTS/blob/main/trabalho_final_audio_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar as bibliotecas necessárias
!pip install SpeechRecognition
!pip install gtts
!pip install pydub
!pip install sounddevice
!pip install pyaudio

!brew install ffmpeg
#!sudo apt-get install ffmeg



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)
/bin/bash: line 1: brew: command not found


In [2]:
# Importar as bibliotecas
import sounddevice as sd
from scipy.io.wavfile import write
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
from datetime import datetime
import pyaudio





OSError: PortAudio library not found

In [ ]:
# Função para capturar áudio
def capture_audio(filename='output.wav', duration=5, fs=44100):
    print("Gravando...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()  # Esperar a gravação terminar
    write(filename, fs, recording)
    print("Gravação concluída")

def convert_to_wav(input_file, output_file):
    try:
        audio = AudioSegment.from_file(input_file)
        audio.export(output_file, format="wav")
        print(f"Arquivo convertido para {output_file}")
    except Exception as e:
        print(f"Erro ao converter arquivo: {e}")

# Função para reconhecer fala
def recognize_speech(filename='output.wav'):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='pt-BR')
            print("Você disse: " + text)
            return text
        except sr.UnknownValueError:
            print("Não consegui entender o áudio")
            return ""
        except sr.RequestError as e:
            print("Erro no serviço de reconhecimento de fala; {0}".format(e))
            return ""

# Função para gerar e salvar frases com TTS
def generate_and_save_tts(text, filename):
    tts = gTTS(text=text, lang='pt')
    tts.save(filename)

# Função para responder com áudio
def respond_with_audio(response_text):
    tts = gTTS(text=response_text, lang='pt')
    tts.save("response.mp3")
    response_audio = AudioSegment.from_file("response.mp3", format="mp3")
    play(response_audio)

# Função principal da URA
def ura():
    audio_files = {
        1: 'option_1.mp3',
        2: 'option_2.mp3',
        3: 'option_3.mp3',
        4: 'option_4.mp3'
    }
    # Loop principal
    while True:
        # Reproduzir boas-vindas e opções
        play(AudioSegment.from_file('welcome.mp3'))
        for i in range(1, 5):
            play(AudioSegment.from_file(audio_files[i]))

        # Capturar áudio do usuário e reconhecer escolha
        print("Aguardando sua escolha...")
        capture_audio()
        convert_to_wav('output.wav', 'output.wav')
        user_choice = recognize_speech('output.wav')  # Arquivo de áudio capturado pelo usuário

        # Identificar e responder à escolha do usuário
        if 'saldo' in user_choice:
            play(AudioSegment.from_file('confirmation_1.mp3'))
        elif 'simulação' in user_choice:
            play(AudioSegment.from_file('confirmation_2.mp3'))
        elif 'atendente' in user_choice:
            play(AudioSegment.from_file('confirmation_3.mp3'))
        elif 'sair' in user_choice:
            play(AudioSegment.from_file('confirmation_4.mp3'))
            break  # Encerra o loop quando o usuário escolhe sair do atendimento
        else:
            print("Opção não reconhecida. Por favor, tente novamente.")
            play(AudioSegment.from_file('retry_option.mp3'))

    # Fim do atendimento
    print("Atendimento encerrado.")

In [ ]:
# Frases de boas-vindas e opções
welcome_text = "Bem-vindo ao atendimento da Empresa Quantum Finance. Por favor, escolha uma das opções a seguir:"
option_texts = [
    "Para consulta ao saldo da conta, diga Saldo da Conta.",
    "Para simulação de compra internacional, diga simulação de compra.",
    "Para falar com um atendente, diga falar com um atendente.",
    "Para sair do atendimento, diga sair."
]

# Frases de confirmação de escolha
confirmation_texts = [
    "Você escolheu a opção 'Consulta ao saldo da conta'. Seu saldo atual é de 2 milhões e zero centavos.",
    "Você escolheu a opção 'Simulação de compra internacional'. No momento não estamos realizando comprars internacionais.",
    "Você escolheu a opção 'Falar com um atendente'. No momento todos os atendentes estão ocupados, tente novamente mais tarde.",
    "Você escolheu a opção 'Sair do atendimento'. Seu atendimento será encerrado."
]

# Frases de confirmação de escolha
retry_text = "Opção não reconhecida. Por favor, tente novamente."



# Gerar e salvar áudios
generate_and_save_tts(welcome_text, 'welcome.mp3')
generate_and_save_tts(retry_text, 'retry_option.mp3')

for idx, text in enumerate(option_texts):
    generate_and_save_tts(text, f'option_{idx+1}.mp3')
for idx, text in enumerate(confirmation_texts):
    generate_and_save_tts(text, f'confirmation_{idx+1}.mp3')

In [ ]:
# Executar a URA
ura()

Aguardando sua escolha...
Gravando...
Gravação concluída
Arquivo convertido para output.wav
Você disse: falar com um atendente


KeyboardInterrupt: 